In [1]:
import dask.bag as db
import math
import pandas as pd
import requests

Helper function that displays all the functions that can be invoked on an object.

In [2]:
def display_object_method(obj):
    object_methods = [method_name for method_name in dir(obj)
                  if callable(getattr(obj, method_name))]
    print(object_methods)

In [3]:
def get_split_words(word_bag):    
    split_words = word_bag.str.split()
    stripped_words = split_words.flatten().map(lambda x: x.strip())
    word_array = stripped_words.filter(lambda x: x!= "")
    return word_array

In [4]:
def get_filtered_words(words):
    stopwords = open("handout/data/stopwords.txt", "r").read().split("\n")
    return words.filter(lambda x: x not in stopwords)

In [5]:
def get_words_for_document(document):
    word_bag = db.read_text(document)
    split_words = get_split_words(word_bag)
    long_words = split_words.filter(lambda x: len(x) < 3)
#     words_without_punctuations = long_words.map(puntuation_stripper)
#     return get_filtered_words(words_without_punctuations)    
    return long_words

In [6]:
def get_IDF(words_by_document):
    unique_words = []
    for words_for_single_document in words_by_document:
        unique_words.append(words_for_single_document.distinct())
    large_bag = db.concat(unique_words)
    frequencies = large_bag.frequencies()
    idf = frequencies.map(lambda x: (x[0], math.log(len(words_by_document)/x[1])))
    return idf

In [7]:
def get_TF_IDF(tf, idf):
    tf = tf.frequencies()
    tf_idf = tf.join(idf, lambda x: x[0], lambda x: x[0]).map(lambda x: (x[0][0], x[0][1] * x[1][1]))
    return tf_idf.filter(lambda x: x[1] > 0).compute()

In [8]:
set_name = "X_small_train.txt"
# set_name = "X_train.txt"
url = 'https://storage.googleapis.com/uga-dsp/project1/files/' + set_name
file_names = pd.read_csv(url, header=None)[0].to_numpy()

In [9]:
file_words = []
for file_name in file_names:
#     file_name = "data/train/" + file_name + ".bytes"
    file_name = "https://storage.googleapis.com/uga-dsp/project1/data/bytes/" + file_name + ".bytes"
    file_words.append(get_words_for_document(file_name))

idf = get_IDF(file_words).compute()
tf_idf = []
for terms in file_words:
    tf_idf.append(get_TF_IDF(terms, idf))

KeyboardInterrupt: 

In [ ]:
print(len(idf))

In [ ]:
print(tf_idf[0])